## Import dependencies

In [1]:
import random
import copy
import logging
import statistics
import numpy as np

# Get the root logger and set its level to DEBUG
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Set up classes

In [2]:
class Squad:
    def __init__(self, name, *unit_pairs):
        self.name = name
        self.unit_pairs = unit_pairs
        self.units = []
        self.spawn_squad()

    def spawn_squad(self):
        self.units = []
        for unit, count in self.unit_pairs:
            self.units.extend([copy.deepcopy(unit) for _ in range(count)])
    
    def reset_squad_health(self):
        for unit in self.units:
            unit.health = unit.max_health
    
    def is_alive(self):
        return any(unit.health > 0 for unit in self.units)

    def get_alive_unit(self):
        for unit in self.units:
            if unit.is_alive():
                return unit
        return None
    
    def total_health(self):
        return sum(unit.health for unit in self.units)

class Weapon:
    def __init__(self, name, attacks, ballistic_skill, strength, armor_piercing, damage):
        self.name = name
        self.attacks = attacks
        self.ballistic_skill = ballistic_skill
        self.strength = strength
        self.armor_piercing = armor_piercing
        self.damage = damage

    def get_attacks(self):
        if callable(self.attacks):
            dice_roll,added_number=self.attacks()
            number_of_attacks=dice_roll+added_number
            logging.debug(f"The method is: {self.get_attacks.__name__}. The amount of damage was: {number_of_attacks}. The dice roll was {dice_roll} and the added number was {added_number}")
            return number_of_attacks
        else:
            return self.attacks
        
    def get_damage(self):
        if callable(self.damage):
            dice_roll,added_number=self.damage()
            number_of_damage=dice_roll+added_number
            logging.debug(f"The method is: {self.get_damage.__name__}. The amount of damage was: {number_of_damage}. The dice roll was {dice_roll} and the added number was {added_number}")
            return number_of_damage
        else:
            return self.damage

class Unit:
    def __init__(self, name, weapons, toughness, armor, max_health, feel_no_pain=None, invulnerable_save=None,):
        self.name = name
        self.weapons = weapons
        self.toughness = toughness
        self.armor = armor
        self.invulnerable_save = invulnerable_save
        self.max_health = max_health
        self.health = max_health
        self.feel_no_pain = feel_no_pain

    def roll_dice(self):
        return random.randint(1, 6)

    def does_it_hit(self, weapon):
        dice_roll = self.roll_dice()
        outcome = dice_roll >= weapon.ballistic_skill
        logging.debug(f"The method is: {self.does_it_hit.__name__}. Weapon: {weapon.name}, Dice Roll: {str(dice_roll)}, Ballistic Skill: {weapon.ballistic_skill}, Outcome: {outcome}")
        return dice_roll >= weapon.ballistic_skill

    def does_it_wound(self, opponent_toughness, weapon):
        roll = self.roll_dice()
        outcome = False
        if weapon.strength == opponent_toughness and roll >= 4:
            outcome = True
        elif weapon.strength > opponent_toughness and roll >= 3:
            outcome = True
        elif weapon.strength >= 2 * opponent_toughness and roll >= 2:
            outcome = True
        elif weapon.strength < opponent_toughness and roll >= 5:
            outcome = True
        elif weapon.strength <= 0.5 * opponent_toughness and roll == 6:
            outcome = True

        logging.debug(f"The method is: {self.does_it_wound.__name__}. Weapon: {weapon.name}, Dice Roll: {str(roll)}, Opponent Toughness: {opponent_toughness}, Outcome: {outcome}")
        return outcome

    def does_it_pass_through_armor(self, weapon):
        if self.invulnerable_save:
            modified_armor = min(self.invulnerable_save, self.armor - weapon.armor_piercing)
        else:
            modified_armor=self.armor - weapon.armor_piercing
        dice_roll = self.roll_dice()
        outcome = dice_roll < modified_armor
        logging.debug(f"The method is: {self.does_it_pass_through_armor.__name__}, Weapon: {weapon.name}, Dice Roll: {str(dice_roll)}, Modified Armor: {modified_armor}, Outcome: {outcome}")
        return outcome

    def feel_no_pain_save(self,damage):
        if self.feel_no_pain:
            remaining_damage=0
            for _ in range(damage):
                dice_roll = self.roll_dice()
                if dice_roll < self.feel_no_pain:
                    remaining_damage += 1
            logging.debug(f"The method is: {self.feel_no_pain_save.__name__}, Unit: {self.name},Incoming damage: {damage}, Remaining damage: {remaining_damage}")
        else:
            remaining_damage=damage
            logging.debug(f"The method is: {self.feel_no_pain_save.__name__}, No feel no pain.,Incoming damage: {damage}, Remaining damage: {remaining_damage}")
        return remaining_damage
    
    def is_alive(self):
        return self.health > 0

In [3]:
# unit_pairs=[(assault_intercessor_ranged, 2)]

# testunits = []
# for unit, count in unit_pairs:
#     # testunits.extend([copy.deepcopy(unit) for _ in range(count)])
#     testunits.extend([unit for _ in range(count)])
#     testunits[0].health = 99
# testunits

## Set up Weapons, Units and Squads

In [4]:
# Variable number of attacks
def roll_dice_plus_variable(dice_number,added_variable):
    return random.randint(1, dice_number),added_variable

# Create weapons
bolter = Weapon(name='bolter', attacks=2, ballistic_skill=3, strength=4, armor_piercing=0, damage=2)
genestealer_claws = Weapon(name='genestealer_claws',attacks=4, ballistic_skill=2, strength=4, armor_piercing=-2, damage=1)
heavy_bolt_pistol = Weapon(name='heavy_bolt_pistol', attacks=1, ballistic_skill=3, strength=4, armor_piercing=-1, damage=1)
astartes_chainsword = Weapon(name='astartes_chainsword', attacks=4, ballistic_skill=3, strength=4, armor_piercing=-1, damage=1)
old_one_eye_strike = Weapon(name='old_one_eye_strike', attacks=6, ballistic_skill=3, strength=14, armor_piercing=-3, damage=lambda: roll_dice_plus_variable(6,1))
old_one_eye_sweep = Weapon(name='old_one_eye_sweep', attacks=12, ballistic_skill=3, strength=6, armor_piercing=-1, damage=1)
macro_plasma_incinerator = Weapon(name='macro_plasma_incinerator', attacks=lambda: roll_dice_plus_variable(6,1), ballistic_skill=3, strength=8, armor_piercing=-3, damage=2)
macro_plasma_incinerator_supercharged = Weapon(name='macro_plasma_incinerator', attacks=lambda: roll_dice_plus_variable(6,1), ballistic_skill=3, strength=9, armor_piercing=-4, damage=3)
storm_bolter = Weapon(name='storm_bolter', attacks=4, ballistic_skill=3, strength=4, armor_piercing=0, damage=1)
onslaught_gatling_gun = Weapon(name='onslaught_gatling_gun', attacks=8, ballistic_skill=3, strength=5, armor_piercing=0, damage=1)




# Create units
assault_intercessor_ranged = Unit(name='assault_intercessor_ranged', weapons=[heavy_bolt_pistol], toughness=4, armor=3,  max_health=2)
assault_intercessor_melee = Unit(name='assault_intercessor_melee', weapons=[heavy_bolt_pistol,astartes_chainsword], toughness=4, armor=3,  max_health=2)
genestealer = Unit(name='Genestealer', weapons=[genestealer_claws], toughness=4, armor=5,  max_health=2, invulnerable_save=5)
old_one_eye=Unit(name='old_one_eye', weapons=[old_one_eye_strike,old_one_eye_sweep], toughness=9, armor=2,  max_health=9, feel_no_pain=5)
redemptor_dreadnought_ranged=Unit(name='redemptor_dreadnought_ranged', weapons=[macro_plasma_incinerator,storm_bolter,onslaught_gatling_gun], toughness=10, armor=2,  max_health=12)
redemptor_dreadnought_ranged_supercharged=Unit(name='redemptor_dreadnought_ranged', weapons=[macro_plasma_incinerator_supercharged,storm_bolter,onslaught_gatling_gun], toughness=10, armor=2,  max_health=12)


assault_intercessor_ranged_squad = Squad('Assault Intercessor Ranged Squad', (assault_intercessor_ranged, 5))
assault_intercessor_melee_squad = Squad('Assault Intercessor Melee Squad', (assault_intercessor_melee, 5))
genestealer_squad = Squad('Genestealer Squad', (genestealer, 10))
old_one_eye_squad = Squad('Old One Eye Squad', (old_one_eye, 1))  # Assuming you want only one Old One Eye in the squad
redemptor_dreadnought_ranged_squad = Squad('Redemptor Dreadnought Ranged Squad', (redemptor_dreadnought_ranged, 1))
redemptor_dreadnought_ranged_supercharged_squad = Squad('Redemptor Dreadnought Ranged Supercharged Squad', (redemptor_dreadnought_ranged_supercharged, 1))

## Set up simulation

In [5]:
class Simulation:
    def __init__(self, attacker_squad, defender_squad, trials=5):
        attacker_squad.spawn_squad()
        defender_squad.spawn_squad()
        self.attacker_squad = attacker_squad
        self.defender_squad = defender_squad
        self.trials = trials
        self.initial_defender_squad_health=defender_squad.total_health()
        self.initial_number_of_defender_units=len(defender_squad.units)


    def run(self):
        damage_results=[]
        health_reduced_results=[]
        death_results=[]
        for _ in range(self.trials):
            logging.debug("\n\n\nStarting a new trial")
            total_damage = 0
            total_health_reduced = 0
            total_deaths = 0
            self.attacker_squad.reset_squad_health()
            self.defender_squad.reset_squad_health()
            for attacker in self.attacker_squad.units:
                for weapon in attacker.weapons:
                    for _ in range(weapon.get_attacks()):
                        if attacker.does_it_hit(weapon):
                            defender = self.defender_squad.get_alive_unit()
                            if defender is None:
                                logging.debug("All units in the squad are dead")
                                break
                            if attacker.does_it_wound(defender.toughness, weapon):
                                if defender.does_it_pass_through_armor(weapon):
                                    damage = weapon.get_damage()
                                    remaining_damage=defender.feel_no_pain_save(damage)
                                    logging.debug(f"Unit: {defender.name}, Health: {defender.health}, Damage: {remaining_damage}")
                                    total_damage += remaining_damage
                                    for _ in range(remaining_damage):
                                        defender.health -= 1
                                        total_health_reduced += 1
                                        logging.debug(f"Reduced one health. total health reduced is: {total_health_reduced}")

                                        if defender.health <= 0:
                                            total_deaths += 1
                                            remaining_units = sum(1 for unit in self.defender_squad.units if unit.health > 0)
                                            initial_number_of_units = sum(1 for unit in self.defender_squad.units)

                                            logging.debug(f"Unit: {defender.name} has died : (  {remaining_units} remaining out of {initial_number_of_units}.")
                                            break
            damage_results.append(total_damage)
            health_reduced_results.append(total_health_reduced)
            death_results.append(total_deaths)
        return {"damage":damage_results,"health_reduced":health_reduced_results,"deaths":death_results,"initial_defender_squad_health":self.initial_defender_squad_health}




## Performance profiling

In [6]:
import cProfile
import pstats

logger.setLevel(logging.INFO)
number_of_trials=10000
def run_many_simulations():
    # for _ in range(number_of_trials):
    simulation = Simulation(redemptor_dreadnought_ranged_squad, old_one_eye_squad, trials=number_of_trials)
    result = simulation.run()

profiler = cProfile.Profile()
profiler.enable()
run_many_simulations()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

# Get the total time
total_time = sum([x.totaltime for x in profiler.getstats()])
logger.info(f'Average time per simulation: {round(total_time/number_of_trials,4)}')

INFO:root:Average time per simulation: 0.0007


         5324546 function calls (5324425 primitive calls) in 0.979 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.979    0.489 /Users/lucastamatescumicrosoft/Code/warhammer-40K-simulator/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517(run_code)
        2    0.000    0.000    0.979    0.489 {built-in method builtins.exec}
        1    0.000    0.000    0.979    0.979 /var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_44186/274802400.py:6(run_many_simulations)
        1    0.089    0.089    0.979    0.979 /var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_44186/1181690119.py:12(run)
   165119    0.105    0.000    0.395    0.000 /var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_44186/918884673.py:70(does_it_hit)
   329943    0.053    0.000    0.384    0.000 /var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_44186/918884673.py:67(roll_

In [7]:
logger.setLevel(logging.DEBUG)
simulation = Simulation(old_one_eye_squad, genestealer_squad, trials=10)
result = simulation.run()
logger.setLevel(logging.INFO)

result

DEBUG:root:


Starting a new trial
DEBUG:root:The method is: does_it_hit. Weapon: old_one_eye_strike, Dice Roll: 4, Ballistic Skill: 3, Outcome: True
DEBUG:root:The method is: does_it_wound. Weapon: old_one_eye_strike, Dice Roll: 6, Opponent Toughness: 4, Outcome: True
DEBUG:root:The method is: does_it_pass_through_armor, Weapon: old_one_eye_strike, Dice Roll: 1, Modified Armor: 5, Outcome: True
DEBUG:root:The method is: get_damage. The amount of damage was: 5. The dice roll was 4 and the added number was 1
DEBUG:root:The method is: feel_no_pain_save, No feel no pain.,Incoming damage: 5, Remaining damage: 5
DEBUG:root:Unit: Genestealer, Health: 2, Damage: 5
DEBUG:root:Reduced one health. total health reduced is: 1
DEBUG:root:Reduced one health. total health reduced is: 2
DEBUG:root:Unit: Genestealer has died : (  9 remaining out of 10.
DEBUG:root:The method is: does_it_hit. Weapon: old_one_eye_strike, Dice Roll: 3, Ballistic Skill: 3, Outcome: True
DEBUG:root:The method is: does_it_wou

{'damage': [13, 10, 4, 31, 18, 26, 14, 14, 10, 12],
 'health_reduced': [9, 8, 4, 15, 9, 13, 11, 9, 7, 7],
 'deaths': [4, 4, 2, 7, 4, 6, 5, 4, 3, 3],
 'initial_defender_squad_health': 20}

In [9]:

def calculate_metrics(result):
    result['wasted_damage'] = [d - h for d, h in zip(result['damage'], result['health_reduced'])]
    result['health_reduced_fraction'] = [h / result['initial_defender_squad_health'] for h in result['health_reduced']]

    average_damage = statistics.mean(result['damage'])
    damage_stdev = statistics.stdev(result['damage'])
    average_health_reduced = statistics.mean(result['health_reduced'])
    health_reduced_stdev = statistics.stdev(result['health_reduced'])
    average_wasted_damage=average_damage-average_health_reduced


    def calculate_quartiles_and_iqr(data, key):
        Q1 = round(np.percentile(data[key], 25),2)
        Q2 = round(np.percentile(data[key], 50),2)
        Q3 = round(np.percentile(data[key], 75),2)
        IQR = Q3 - Q1
        return Q1, Q2, Q3, IQR

    Q1_damage, Q2_damage, Q3_damage, IQR_damage = calculate_quartiles_and_iqr(result, 'damage')
    Q1_health_reduced, Q2_health_reduced, Q3_health_reduced, IQR_health_reduced = calculate_quartiles_and_iqr(result, 'health_reduced')
    Q1_wasted_damage, Q2_wasted_damage, Q3_wasted_damage, IQR_wasted_damage = calculate_quartiles_and_iqr(result, 'wasted_damage')
    Q1_deaths, Q2_deaths, Q3_deaths, IQR_deaths = calculate_quartiles_and_iqr(result, 'deaths')
    Q1_health_reduced_fraction, Q2_health_reduced_fraction, Q3_health_reduced_fraction, IQR_health_reduced_fraction = calculate_quartiles_and_iqr(result, 'health_reduced_fraction')

    # print(f"Median damage: {Q2_damage} ({Q1_damage} <> {Q3_damage})")
    # print(f"Median health reduced: {Q2_health_reduced} ({Q1_health_reduced} <> {Q3_health_reduced})")
    # print(f"Median wasted damage: {Q2_wasted_damage} ({Q1_wasted_damage} <> {Q3_wasted_damage})")
    # print(f"Median deaths: {Q2_deaths} ({Q1_deaths} <> {Q3_deaths})")
    # print(f"Median health reduced fraction: {Q2_health_reduced_fraction} ({Q1_health_reduced_fraction} <> {Q3_health_reduced_fraction})")
    # return values as a dict
    return {
        'average_damage': average_damage,
        'damage_stdev': damage_stdev,
        'average_health_reduced': average_health_reduced,
        'health_reduced_stdev': health_reduced_stdev,
        'average_wasted_damage': average_wasted_damage,
        'Q1_damage': Q1_damage,
        'Q2_damage': Q2_damage,
        'Q3_damage': Q3_damage,
        'IQR_damage': IQR_damage,
        'Q1_health_reduced': Q1_health_reduced,
        'Q2_health_reduced': Q2_health_reduced,
        'Q3_health_reduced': Q3_health_reduced,
        'IQR_health_reduced': IQR_health_reduced,
        'Q1_wasted_damage': Q1_wasted_damage,
        'Q2_wasted_damage': Q2_wasted_damage,
        'Q3_wasted_damage': Q3_wasted_damage,
        'IQR_wasted_damage': IQR_wasted_damage,
        'Q1_deaths': Q1_deaths,
        'Q2_deaths': Q2_deaths,
        'Q3_deaths': Q3_deaths,
        'IQR_deaths': IQR_deaths,
        'Q1_health_reduced_fraction': Q1_health_reduced_fraction,
        'Q2_health_reduced_fraction': Q2_health_reduced_fraction,
        'Q3_health_reduced_fraction': Q3_health_reduced_fraction,
        'IQR_health_reduced_fraction': IQR_health_reduced_fraction,
    }


## Run comparisons

In [10]:
import pandas as pd

# Create a list of all squads
squads = [assault_intercessor_ranged_squad, assault_intercessor_melee_squad, genestealer_squad,old_one_eye_squad,redemptor_dreadnought_ranged_squad,redemptor_dreadnought_ranged_supercharged_squad]
# squads = [old_one_eye_squad,redemptor_dreadnought_ranged_squad,redemptor_dreadnought_ranged_supercharged_squad,genestealer_squad]

# Create an empty list to store the results
results = []
colour=[]

trials=1000
logger.info("Total number of trials: " + str(trials*len(squads)*len(squads)))

# Iterate over all pairs of squads
for i in range(len(squads)):
    for j in range(len(squads)):
        # Run a simulation for the current pair of squads
        simulation = Simulation(squads[i], squads[j], trials=trials)
        result = simulation.run()
        metrics=calculate_metrics(result)

        # Store the result in the results list
        results.append({
            'attacker': squads[i].name,
            'defender': squads[j].name,
            'damage': f"{metrics['Q2_damage']} ({metrics['Q1_damage']} <> {metrics['Q3_damage']})",
            'IQR_damage': f"{metrics['IQR_damage']}",
            'health_reduced': f"{metrics['Q2_health_reduced']} ({metrics['Q1_health_reduced']} <> {metrics['Q3_health_reduced']})",
            'IQR_health_reduced': f"{metrics['IQR_health_reduced']}",
            'wasted_damage': f"{metrics['Q2_wasted_damage']} ({metrics['Q1_wasted_damage']} <> {metrics['Q3_wasted_damage']})",
            'deaths': f"{metrics['Q2_deaths']} ({metrics['Q1_deaths']} <> {metrics['Q3_deaths']})",
            'health_reduced_fraction': f"{metrics['Q2_health_reduced_fraction']} ({metrics['Q1_health_reduced_fraction']} <> {metrics['Q3_health_reduced_fraction']})",
        })
        colour.append({
            'attacker': squads[i].name,
            'damage_colour':metrics['Q2_damage'],
            'damage_colour': metrics['Q2_damage'],
            'health_reduced_colour': metrics['Q2_health_reduced'],
            'wasted_damage_colour': metrics['Q2_wasted_damage'],
            'deaths_colour': metrics['Q2_deaths'],
            'health_reduced_fraction_colour': metrics['Q2_health_reduced_fraction'],
            'IQR_damage_colour': metrics['IQR_damage'],
            'IQR_health_reduced_colour': metrics['IQR_health_reduced'],
        })
        

# Convert the results list to a DataFrame
df = pd.DataFrame(results)
df_colour = pd.DataFrame(colour)

# df=df[df['attacker'] == 'Old One Eye Squad']
# df_colour=df_colour[df_colour['attacker'] == 'Old One Eye Squad']

pd.set_option('display.max_colwidth', 30)
# Normalize each column to range [0, 1], grouped by "attacker"
normalized_df_colour = df_colour.groupby('attacker').transform(lambda x: (x - x.min()) / (x.max() - x.min()))

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def color_map(val, reverse=False):
    cmap = plt.get_cmap('RdYlGn')  # Get the colormap
    if reverse:
        cmap = cmap.reversed()  # Reverse the colormap
    norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalize the values to range [0, 1]
    
    rgba = cmap(norm(val))  # Get the rgba value from the colormap
    return f'background-color: {mcolors.rgb2hex(rgba)}'  # Convert rgba to hex and return as CSS

# Create a Styler object from df
styled_df = df.style

# Apply the color map to only 'health_reduced_fraction' column in df using the values in color_df
styled_df = styled_df.apply(lambda s: normalized_df_colour['damage_colour'].apply(lambda x: color_map(x, reverse=False)), subset=['damage'])
styled_df = styled_df.apply(lambda s: normalized_df_colour['health_reduced_colour'].apply(lambda x: color_map(x, reverse=False)), subset=['health_reduced'])
styled_df = styled_df.apply(lambda s: normalized_df_colour['wasted_damage_colour'].apply(lambda x: color_map(x, reverse=True)), subset=['wasted_damage'])
styled_df = styled_df.apply(lambda s: normalized_df_colour['health_reduced_fraction_colour'].apply(lambda x: color_map(x, reverse=False)), subset=['health_reduced_fraction'])
styled_df = styled_df.apply(lambda s: normalized_df_colour['IQR_damage_colour'].apply(lambda x: color_map(x, reverse=True)), subset=['IQR_damage'])
styled_df = styled_df.apply(lambda s: normalized_df_colour['IQR_health_reduced_colour'].apply(lambda x: color_map(x, reverse=True)), subset=['IQR_health_reduced'])

## show full width
# styled_df = styled_df.set_properties(**{'white-space': 'nowrap', 'overflow': 'hidden'})

# hide when text is too long
styled_df = styled_df.set_properties(**{'max-width': '150px', 'white-space': 'nowrap', 'overflow': 'hidden'})

# #add scrollbar when text is too long
# styled_df = styled_df.set_properties(**{'max-width': '150px', 'white-space': 'nowrap', 'overflow': 'auto'})

styled_df

/var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_44186/63010485.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
INFO:root:Total number of trials: 36000


,attacker,defender,damage,IQR_damage,health_reduced,IQR_health_reduced,wasted_damage,deaths,health_reduced_fraction
0,Assault Intercessor Ranged Squad,Assault Intercessor Ranged Squad,1.0 (0.0 <> 1.0),1.0,1.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.1 (0.0 <> 0.1)
1,Assault Intercessor Ranged Squad,Assault Intercessor Melee Squad,1.0 (0.0 <> 1.0),1.0,1.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.1 (0.0 <> 0.1)
2,Assault Intercessor Ranged Squad,Genestealer Squad,1.0 (0.0 <> 2.0),2.0,1.0 (0.0 <> 2.0),2.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 1.0),0.05 (0.0 <> 0.1)
3,Assault Intercessor Ranged Squad,Old One Eye Squad,0.0 (0.0 <> 0.0),0.0,0.0 (0.0 <> 0.0),0.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0)
4,Assault Intercessor Ranged Squad,Redemptor Dreadnought Ranged Squad,0.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.08)
5,Assault Intercessor Ranged Squad,Redemptor Dreadnought Ranged Supercharged Squad,0.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 1.0),1.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.08)
6,Assault Intercessor Melee Squad,Assault Intercessor Ranged Squad,4.0 (3.0 <> 5.0),2.0,4.0 (3.0 <> 5.0),2.0,0.0 (0.0 <> 0.0),2.0 (1.0 <> 2.0),0.4 (0.3 <> 0.5)
7,Assault Intercessor Melee Squad,Assault Intercessor Melee Squad,4.0 (3.0 <> 5.0),2.0,4.0 (3.0 <> 5.0),2.0,0.0 (0.0 <> 0.0),2.0 (1.0 <> 2.0),0.4 (0.3 <> 0.5)
8,Assault Intercessor Melee Squad,Genestealer Squad,5.0 (4.0 <> 7.0),3.0,5.0 (4.0 <> 7.0),3.0,0.0 (0.0 <> 0.0),2.0 (2.0 <> 3.0),0.25 (0.2 <> 0.35)
9,Assault Intercessor Melee Squad,Old One Eye Squad,1.0 (0.0 <> 2.0),2.0,1.0 (0.0 <> 2.0),2.0,0.0 (0.0 <> 0.0),0.0 (0.0 <> 0.0),0.11 (0.0 <> 0.22)
